# Mining Processes From Smart Contracts

In [ ]:
from web3 import Web3

In [ ]:
w3 = Web3(Web3.WebsocketProvider("ws://127.0.0.1:8545", websocket_kwargs={'timeout': 60}))
w3.isConnected()

In [ ]:
event_identifiers = {}
events = {
    b"Created(bytes)": "",
    b"Started()": "",
    b"Finished(bytes)": "",
    b"TaskRequest(uint8,bytes)": "",
    b"TaskResponse(uint8,bytes)": "",
    b"SubChoreographyCreated(uint8,bytes,address)": "",
    b"SubChoreographyFinished(uint8,bytes,bool)": "",
    b"ThrewTrigger(uint8,bytes)": "",
    b"CaughtTrigger(uint8,bytes,address)": "",
    b"UncaughtTrigger(uint8,bytes,address)": "",
    b"ParticipantRegistered(address,uint8)": "",
    b"Interrupted()": "",
    b"Interrupting()": "",
    b"ChoreographyCreated(address,bytes)": "",
    b"DebugFlowExecuted(uint8)": "",
    b"DebugTokens(uint256)": ""
}

In [ ]:
import sha3
for key in events.keys():
    event_hash = sha3.keccak_256()
    event_hash.update(key)
    hash_sum = event_hash.hexdigest()
    events[key] = hash_sum
    event_identifiers[hash_sum] = key

In [ ]:
external_contracts = []

In [ ]:
# Contract addresses extracted w/ ethereum-etl

rental_addresses = [
    "..."
]

In [ ]:
from web3.utils.events import get_event_data

def get_events_for_contract(address):
    contract_filter = w3.eth.filter({
        "fromBlock": "0x0",
        "toBlock": "latest",
        "address": w3.toChecksumAddress(address)
    })
    event_list = contract_filter.get_all_entries()
    contract_events = []
    contract_transaction_events = []
    
    myLogs = w3.eth.getFilterLogs(contract_filter.filter_id)
    event_prefix = ''

    for event in event_list:
        trace_event = {}
        trace_event_attribute = {}
        block_number = event["blockNumber"]
        block = w3.eth.getBlock(block_number)
        block_time = block["timestamp"]
        trace_event["timestamp"] = block_time
        trace_event_attribute["timestamp"] = block_time
        
        topic = event["topics"][0].hex()[2:] # remove 0x from hex string
        if topic in event_identifiers.keys():
            trace_event["event"] = event_identifiers[topic]
            if event_identifiers[topic] == b"SubChoreographyCreated(uint8,bytes,address)":
                external_contracts.append(str(w3.toChecksumAddress('0x'+event['data'][154:194])))

            if str(w3.toChecksumAddress(address)) in external_contracts:
                event_prefix = 'external_'
            else:
                event_prefix = ''

            if event_identifiers[topic] == b'TaskRequest(uint8,bytes)':
                trace_event_attribute["event"] = event_prefix + event["data"][65:66]
                contract_transaction_events.append(trace_event_attribute)      
        else:
            trace_event["event"] = topic
        contract_events.append(trace_event)
    return(contract_events, contract_transaction_events)

In [ ]:
control_flow_traces = []
task_attribute_traces = []

for address in rental_addresses:
    events, attributes = get_events_for_contract(address)
    control_flow_traces.append(events)
    task_attribute_traces.append(attributes)

In [ ]:
import uuid
from opyenxes.factory.XFactory import XFactory
from opyenxes.id.XIDFactory import XIDFactory
from opyenxes.data_out.XesXmlSerializer import XesXmlSerializer

## Log With Lifecycle Events

In [ ]:
log = XFactory.create_log()
for contract_events in control_flow_traces:
    trace_id = 0
    trace = XFactory.create_trace()

    id_attribute = XFactory.create_attribute_id("id", str(uuid.uuid4()))
    trace.get_attributes()["id"] = id_attribute
    trace_id = trace_id + 1

    for event in contract_events:
        event_descriptor = event["event"]  
        log_event = XFactory.create_event()
        timestamp_int = event["timestamp"]
        timestamp_attribute = XFactory.create_attribute_timestamp(
            "time:timestamp", timestamp_int)
        activity_attribute = XFactory.create_attribute_literal(
            "concept:name", event_descriptor)
        log_event.get_attributes(
        )["timestamp"] = timestamp_attribute
        log_event.get_attributes()["Activity"] = activity_attribute
        trace.append(log_event)

    log.append(trace)

with open("event_log_control_flow.xes", "w") as file:
    XesXmlSerializer().serialize(log, file)

## Log With Task Event Names

In [ ]:
log = XFactory.create_log()
for contract_events in task_attribute_traces:
    trace_id = 0
    trace = XFactory.create_trace()

    id_attribute = XFactory.create_attribute_id("id", str(uuid.uuid4()))
    trace.get_attributes()["id"] = id_attribute
    trace_id = trace_id + 1

    for event in contract_events:
        event_descriptor = event["event"]  
        log_event = XFactory.create_event()
        timestamp_int = event["timestamp"]
        timestamp_attribute = XFactory.create_attribute_timestamp(
            "time:timestamp", timestamp_int)
        activity_attribute = XFactory.create_attribute_literal(
            "concept:name", event_descriptor)
        log_event.get_attributes(
        )["timestamp"] = timestamp_attribute
        log_event.get_attributes()["Activity"] = activity_attribute
        trace.append(log_event)

    log.append(trace)

with open("event_log_task_ids.xes", "w") as file:
    XesXmlSerializer().serialize(log, file)

# Process Mining

## Alpha Miner

In [ ]:
from pm4py.algo.discovery.alpha import factory as alpha_miner
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.visualization.petrinet import factory as vis_factory

log = xes_importer.import_log('event_log_task_ids.xes')
net, initial_marking, final_marking = alpha_miner.apply(log)

gviz = vis_factory.apply(net, initial_marking, final_marking)
vis_factory.view(gviz)

## Inductive Miner

In [ ]:
import os
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.algo.discovery.inductive import factory as inductive_miner
from pm4py.visualization.process_tree import factory as pt_vis_factory

og = xes_importer.import_log(os.path.join("event_log_task_ids.xes"))
net, initial_marking, final_marking = inductive_miner.apply(log)

gviz = vis_factory.apply(net, initial_marking, final_marking)
vis_factory.view(gviz)

## Heuristic Miner

In [ ]:
import os
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.algo.discovery.heuristics import factory as heuristics_miner
from pm4py.visualization.heuristics_net import factory as hn_vis_factory

log_path = os.path.join("event_log_task_ids.xes")
log = xes_importer.apply(log_path)
heu_net = heuristics_miner.apply_heu(log, parameters={"dependency_thresh": 0.95})

gviz = hn_vis_factory.apply(heu_net)
hn_vis_factory.view(gviz)